In [ ]:
# !pip install -U sentence-transformers
# !pip install -U langchain-community
# !pip install -U openai

In [ ]:
import langchain
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from openai import AzureOpenAI,OpenAI
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import numpy as np
import pandas as pd

In [ ]:
import os
!export AZURE_OPEN_API_KEY=""
AZURE_OPEN_API_KEY = "bbf9640b5603473b8200e4bb9c0d03d3"
os.environ["AZURE_OPEN_API_KEY"] = AZURE_OPEN_API_KEY

In [ ]:
loader = WebBaseLoader("https://developer.bigfix.com/relevance/guide/client/")

In [ ]:
loader

In [ ]:
model = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [ ]:
retriever=index.vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents('Give the processor information')
# AZURE_OPEN_API_KEY: "bbf9640b5603473b8200e4bb9c0d03d3"

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='System Information\nTo query basic information about clients.\nFolders and Files\nTo get information about what files and folders exist, and the contents of those files.\nRegistry\nTo query information in the Windows Registry.\nNetwork\nTo get information about ports that are open on clients.\nApplications\nTo get information about installed applications.\nServices\nTo query information about running services.\nUsers\nTo get information about users that are logged in.\nEvent Log\nTo get information from the Windows Event Logs.\nWMI\nTo query Windows Management Instrumentation.\n\n\n\n\n\nRelevance\n\n\nHome\n\n\nGuides\n\n\nCore Inspectors\n\n\nClient Relevance\n\n\nSystem Information\n\n\nFiles\n\n\nRegistry\n\n\nNetwork\n\n\nApplications\n\n\nServices\n\n\nUsers\n\n\nEvent Log\n\n\nWMI\n\n\n\n\nSession Relevance\n\n\nError Messages\n\n\n\n\nTutorial\n\n\nReference\n\n\nInspector Search\n\n\nOnline Evaluator\n\n\nExamples\n\n\n\n\n\n\n\n\n\nTerms of use', metad

In [ ]:
def find_answer(question):
    # Use the retriever to get relevant documents based on the question
    docs = retriever.get_relevant_documents(question)
    context = ""

    # Combine the content of all relevant documents into one context string
    for doc in docs:
        context += f"{doc.page_content}\n"

    # Prepare the prompt for the language model
    # prompt = f"Using the following pieces of context:\n{context}\n Please Answer the following question and please generate answer only, no explanations are needed:"

    prompt = f"Using the following pieces of context:\n{context}\n Please Answer the following question.I will be passing the output as relevance expression to HCL BigFix utility. So please do not generate anything other than relevance code "

    client = AzureOpenAI(
        api_key = AZURE_OPEN_API_KEY,
        api_version="2023-03-15-preview",
        azure_endpoint="https://hclsparcaimodel.openai.azure.com/"
    )
    response = client.chat.completions.create(
    model="sparcgpt4-32",
     messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": question}
    ],
    temperature= 0.5,
    max_tokens= 100,
    )
    data = response.choices[0].message.content
    # Split the response at '\n\n'
    split_data = data.split('\n\n')

    # Check if the list has at least two elements
    if len(split_data) >= 2:
        # If it does, return the second element
        data = split_data[1]
    # data = data.strip()
    # # Remove extra spaces within the string
    # data = ' '.join(data.split())
    # Extract and return the content from the response
    return data

In [ ]:
question = 'Give the processor information'
data = find_answer(question)
print(data)

"(name of it, speed of it, number of processors) of processors"


In [ ]:
question = 'find the speed of processor '
data = find_answer(question)
print(data)

"speed of main processor"


In [ ]:
question = 'find unique processor names '
data = find_answer(question)
print(data)

```
unique values of names of processors
```


In [ ]:
question = 'find name of operating system '
data = find_answer(question)
print(data)

"name of operating system"


In [ ]:
question = 'find family names of processor'
data = find_answer(question)
print(data)

"family name of processor"


In [ ]:
df = pd.read_excel('/content/documentation_ml_ready.xlsx')
df.shape

(430, 5)

In [ ]:
# Apply the find_answer function to the 'question' column
answers = df['Description'].apply(find_answer)

# Assign the answers to a new column 'RAG_relavance'
df['RAG_relavance'] = answers
# # Save the updated DataFrame to a new Excel file
df.to_excel('updated_dataset.xlsx', index=False)
df.head()

## EVALUATION

In [ ]:
df = pd.read_excel('/content/New_RAG_relavance.xlsx')
df.shape

(430, 6)

In [ ]:
(df['Relevance']==df['RAG_relavance']).sum()

6

In [ ]:
df.head()

,Description,description_prompts,Relevance,text,task,RAG_relavance
0,Give the processor information,Produce BigFix relevance code to address the g...,vendor names of processors,<s>[INST]<<SYS>>Produce BigFix relevance code ...,relevance,```\nname of processor\n```\nThis will return ...
1,find the speed of processor,Formulate BigFix relevance code to match the p...,speeds of processors,<s>[INST]<<SYS>>Formulate BigFix relevance cod...,relevance,```\nname of current processor\n```
2,find unique processor names,Design a relevance clause in accordance with t...,unique values of vendor names of processors,<s>[INST]<<SYS>>Design a relevance clause in a...,relevance,```\nunique values of names of processors\n```
3,find name of operating system,Design a relevance clause in accordance with t...,name of operating system,<s>[INST]<<SYS>>Design a relevance clause in a...,relevance,```\nname of operating system\n```
4,find family names of processor,Generate BigFix relevance rules based on the p...,family names of processors,<s>[INST]<<SYS>>Generate BigFix relevance rule...,relevance,"""family name of processors"""


In [ ]:
df.columns

Index(['Description', 'description_prompts', 'Relevance', 'text', 'task',
       'RAG_relavance'],
      dtype='object')

In [ ]:
def extract_content(data):
    # Split the string at the backticks
    split_data = data.split('```')

    # Check if the list has at least three elements (before, inside, after the backticks)
    if len(split_data) >= 3:
        # If it does, return the second element (inside the backticks)
        content = split_data[1]
    else:
        # If it doesn't, return the whole string
        content = data

    # Remove leading and trailing whitespace
    content = content.strip()

    return content

In [ ]:
# Assume df is your DataFrame and 'column_name' is the name of the column
df['RAG_relavance'] = df['RAG_relavance'].apply(extract_content)

In [ ]:
# df.to_excel('RAG_dataset.xlsx', index=False)

In [ ]:
df.head(10)

,Description,description_prompts,Relevance,text,task,RAG_relavance
0,Give the processor information,Produce BigFix relevance code to address the g...,vendor names of processors,<s>[INST]<<SYS>>Produce BigFix relevance code ...,relevance,name of processor
1,find the speed of processor,Formulate BigFix relevance code to match the p...,speeds of processors,<s>[INST]<<SYS>>Formulate BigFix relevance cod...,relevance,name of current processor
2,find unique processor names,Design a relevance clause in accordance with t...,unique values of vendor names of processors,<s>[INST]<<SYS>>Design a relevance clause in a...,relevance,unique values of names of processors
3,find name of operating system,Design a relevance clause in accordance with t...,name of operating system,<s>[INST]<<SYS>>Design a relevance clause in a...,relevance,name of operating system
4,find family names of processor,Generate BigFix relevance rules based on the p...,family names of processors,<s>[INST]<<SYS>>Generate BigFix relevance rule...,relevance,"""family name of processors"""
5,find version of Operating system,Compose a relevance query for the stated purpo...,"(name of it, version of it) of operating system",<s>[INST]<<SYS>>Compose a relevance query for ...,relevance,relevance version of operating system
6,find the architecture of operating system,Construct a BigFix relevance statement that fi...,"(name of it, architecture of it) of operating ...",<s>[INST]<<SYS>>Construct a BigFix relevance s...,relevance,name of operating system
7,Determine the Operating system,Produce relevance code to address the given de...,windows of operating system,<s>[INST]<<SYS>>Produce relevance code to addr...,relevance,name of operating system
8,Give the Mounted disk drive information,Develop BigFix relevance language for the give...,"(free spaces of it, names of it) of drives",<s>[INST]<<SYS>>Develop BigFix relevance langu...,relevance,names of drives
9,find uptime of OS,Produce BigFix relevance code to address the g...,uptime of operating system,<s>[INST]<<SYS>>Produce BigFix relevance code ...,relevance,operating system uptime
